# Amazon Bedrock에서 Claude3.5 Sonnet 모델을 사용해 주어진 토픽에 대한 파워포인트 슬라이드를 만들어보기

### 환경 설정

#### 참조:  Bedrock 호출
- 참고: https://docs.anthropic.com/en/api/claude-on-amazon-bedrock


In [1]:
# pip install boto3

In [2]:
%load_ext autoreload
%autoreload 2

## 1: Chaninig Prompt를 사용해 파워포인트 슬라이드 생성하기

- 목표: 최소한의 API 호출로 Chaining Prompt를 사용하여 20장의 파워포인트 슬라이드를 생성하기
- 아이디어: 파워포인트 슬라이드 20장을 구성하기 위해 3번의 Bedrock invoke_model API를 호출한다. 
    - (1) Initial prompt를 사용해 전체 아웃라인 짜기 - 즉, 1번째 Bedrock invoke_model API를 호출에서는 토픽에 맞는 전체 개요를 만든다.
    - (2) Slide prompt를 사용해 1번에서 짠 아웃라인에 맞춰서 디테일한 내용을 불렛 포인트로 정리한 슬라이드를 각각 만든다 - 즉, 2번째 호출에서는 1~10번째 슬라이드의 개요를, 3번째 호출에서는 11~20번째 슬라이드 개요만을 사용해 각 슬라이드에 대한 내용을 자세하게 생성한다.

### 템플릿 파일 기술

In [3]:
from util.v2_prompt_util import *

# 템플릿 파일의 경로를 지정
prompt_folder = "prompt_template"
outline_prompt_file_name = "v02_outline_prompt_template.txt"

### Outline prompt template 로딩 및 토픽 추가

In [4]:
from jinja2 import Template

prompt_file_path = os.path.join(prompt_folder, outline_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
outline_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(outline_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Analyze the given topic thoroughly to write in the provided <output_format> JSON format.
Return only the JSON object without additional tags or explanations.

<context>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</context>

<instruction>
1. Title:
   - Create a compelling and concise main title that captures the essence of the presentation.

2. Slides:
   - Generate 20 slides, including an opening and a closing slide.
   - For each slide:
     - Assign a slide_number (01 to 20).
     - Create a clear and informative slide_title.
     
3. Content Guidelines:
   - Ensure a logical flow and structure throughout the presentation.
   - Include an introduction slide (slide 02) and a conclusion slide (slide 19).   

4. Common Points:
   - Include the most important ke

### 전체 프레젠테이션의 Outline을 만드는 prompt 실행

In [5]:
print("\n## Generating Overall Outline by Claude Sonnet3.5:\n")
outline, input_tokens, output_tokens = invoke_claude(outline_prompt)

print(outline)

print("\n" + "="*50 + "\n")
print(f"Input tokens: {input_tokens}, Output tokens: {output_tokens}")
print("\n" + "="*50 + "\n")


## Generating Overall Outline by Claude Sonnet3.5:

{
    "title": "2024 파리 올림픽 폐막식 성공 예측: 미래를 향한 화려한 피날레",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐막식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐막식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요 및 주요 하이라이트"
        },
        {
            "slide_number": "04",
            "slide_title": "역대 올림픽 폐막식 성공 사례 분석"
        },
        {
            "slide_number": "05",
            "slide_title": "파리 2024 폐막식 기획 및 준비 과정"
        },
        {
            "slide_number": "06",
            "slide_title": "폐막식 장소 및 인프라 준비 상황"
        },
        {
            "slide_number": "07",
            "slide_title": "문화적 요소: 프랑스 문화와 세계 문화의 조화"
        },
        {
            "slide_number": "08",
            "slide_title": "기술 혁신: 폐막식에 도입될 최신 기술"
        },
        {
          

In [6]:
# Outline에서 슬라이드 타이틀만 보기
print_slide_titles(outline)

프레젠테이션 제목: 2024 파리 올림픽 폐막식 성공 예측: 미래를 향한 화려한 피날레

슬라이드 01: 2024 파리 올림픽 폐막식 성공 예측
슬라이드 02: 소개: 파리 올림픽의 의미와 폐막식의 중요성
슬라이드 03: 파리 2024 올림픽 개요 및 주요 하이라이트
슬라이드 04: 역대 올림픽 폐막식 성공 사례 분석
슬라이드 05: 파리 2024 폐막식 기획 및 준비 과정
슬라이드 06: 폐막식 장소 및 인프라 준비 상황
슬라이드 07: 문화적 요소: 프랑스 문화와 세계 문화의 조화
슬라이드 08: 기술 혁신: 폐막식에 도입될 최신 기술
슬라이드 09: 참여 예상 국가 및 선수단 규모
슬라이드 10: 안전 및 보안 대책: 테러 위협 대응 방안
슬라이드 11: 환경 친화적 폐막식: 지속 가능성 전략
슬라이드 12: 미디어 커버리지 및 글로벌 시청률 예측
슬라이드 13: 폐막식 공연 프로그램 하이라이트
슬라이드 14: 참여 아티스트 및 유명 인사 소개
슬라이드 15: 디지털 참여: 온라인 및 소셜 미디어 전략
슬라이드 16: 경제적 영향: 폐막식의 경제 효과 분석
슬라이드 17: 도전 과제 및 극복 방안
슬라이드 18: 성공 지표 및 평가 기준
슬라이드 19: 결론: 파리 2024 폐막식 성공 전망
슬라이드 20: 질의응답 및 토론


### Slide prompt template 로딩

In [7]:
# Slide Prompt template
prompt_file_name = "v03_slide_prompt_template.txt"

slide_prompt_file_path = os.path.join(prompt_folder, prompt_file_name)

template_content = load_template_from_file(slide_prompt_file_path)
slide_prompt_template = Template(template_content)
slide_prompt = slide_prompt_template.render()

print("\n## Slide prompt template...")
print(slide_prompt)



## Slide prompt template...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this  of the , Generate detailed content for slides  to .
For each slide, include:
1. Slide title
2. Main content points (3-5 bullet points)
3. Relevant data or examples
4. Suggesion for visuals (e.g., image, chart, graph)
5. A brief speaker note (1-2 sentences)


### 만들어진 슬라이드를 저장할 결과 폴더 생성

In [8]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### Slide prompt template을 기반으로 슬라이드 1 - 10 번을 생성할 Slide prompt 생성

In [9]:
from util.v2_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"

# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
print("\n## Generating prompt for creating the content of slides 1-10...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 1, 10)
print(slide_prompt)

# 1 - 10번째 슬라이드 생성
print("\n## Generating content for slides 1-10...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 1, 10, 1)


## Generating prompt for creating the content of slides 1-10...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐막식 성공 예측: 미래를 향한 화려한 피날레",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐막식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐막식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요 및 주요 하이라이트"
        },
        {
            "slide_number": "04",
            "slide_title": "역대 올림픽 폐막식 성공 사례 분석"
        },
        {
            "slide_number": "05",
            "slide_title": "파리 2024 폐막식 기획 및 준비 과정"
        },
        {
            "slide_number": "06",
            "slide_titl

### Slide prompt template을 기반으로 슬라이드 11 - 20 번을 생성할 Slide prompt 생성

In [10]:
from util.v2_prompt_util import create_slide_prompt, generate_slide_content
# topic = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"

# slide_prompt_list = create_slide_prompt(outline, slide_prompt_template, topic, include_outline=include_outline, max_slide_num=20)
print("\nGenerating prompt for creating the content of slides 11-20...")
slide_prompt = create_slide_prompt(slide_prompt_template, TOPIC, outline, 11, 20)
print(slide_prompt)

# 11 - 20번째 슬라이드 생성
print("\nGenerating content for slides 11-20...")
include_outline = False
slides_1_10 = generate_slide_content(slide_prompt, outline, output_directory, include_outline, 11, 20, 2)


Generating prompt for creating the content of slides 11-20...
You are an expert in creating impactful and professional PowerPoint presentations.
Your task is to create detailed content for a specific slide in 20-slide presentation in Korean, ensuring a logical flow and comprehensive coverage of the topic.

Based on this {
    "title": "2024 파리 올림픽 폐막식 성공 예측: 미래를 향한 화려한 피날레",
    "slides": [
        {
            "slide_number": "01",
            "slide_title": "2024 파리 올림픽 폐막식 성공 예측"
        },
        {
            "slide_number": "02",
            "slide_title": "소개: 파리 올림픽의 의미와 폐막식의 중요성"
        },
        {
            "slide_number": "03",
            "slide_title": "파리 2024 올림픽 개요 및 주요 하이라이트"
        },
        {
            "slide_number": "04",
            "slide_title": "역대 올림픽 폐막식 성공 사례 분석"
        },
        {
            "slide_number": "05",
            "slide_title": "파리 2024 폐막식 기획 및 준비 과정"
        },
        {
            "slide_number": "06",
            "slide_title"

### 두 차례 호출로 만들어졌던 슬라이드 내용을 한 파일에 최종적으로 모으기

In [11]:
# Merge results
merge_files(output_directory)

All files have been merged into 'final_result.txt' in the directory './results/2024-08-21-12-47-33'.


## 2: Zero shot prompting 이지만 Role-based prompting, Task-specific instructions, Content structuring, Output constraints를 사용해서 파워포인트를 생성하기
: Structure the presentation을 미리 prompt (presentation_prompt) 에 써주고, 각 슬라이드의 아웃풋 토큰 개수에 제한을 두기

### 프롬프트 로딩

In [12]:
from jinja2 import Template

all_prompt_file_name = "v01_all_prompt_template.txt"

prompt_file_path = os.path.join(prompt_folder, all_prompt_file_name)

template_content = load_template_from_file(prompt_file_path)
PROMPT_TEMPLATE = Template(template_content)

TOPIC = "Paris 2024 Olympics Cloasing Ceremony Success Prediction"
presentation_prompt = PROMPT_TEMPLATE.render(topic=TOPIC)
# print_json(prompt)
print(presentation_prompt)

You are an expert in creating impactful and professional PowerPoint presentations. 
Create a detailed outline for a 20-slide presentation in Korean, 
ensuring a logical flow and comprehensive coverage of the topic.

<topic>
Paris 2024 Olympics Cloasing Ceremony Success Prediction
</topic>

<instruction>
For each slide, provide:
1. A clear and concise title
2. 3-5 bullet points of key information
3. Any relevant data or statistics (if applicable)

Structure the presentation as follows:
- Slide 1: Introduction
- Slides 2-3: Background and Context
- Slides 4-7: Main Points and Analysis
- Slides 8-9: Predictions and Potential Outcomes
- Slide 10: Conclusion and Key Takeaways

Present the content for all 20 slides in a clear, organized manner. 
The number of output tokens for each slide must not exceed 100
</instruction>


### 생성한 슬라이드 결과를 저장할 폴더 생성

In [13]:
from datetime import datetime
import os

currentDateAndTime = datetime.now()
currentTime = currentDateAndTime.strftime("%Y-%m-%d-%H-%M-%S")
# print("The current time is", currentTime)

output_directory = "./results"
output_directory = os.path.join(output_directory, currentTime)
os.makedirs(output_directory, exist_ok=True)

### 슬라이드 생성

In [15]:
from util.v2_prompt_util import generate_all_slide_content

generate_all_slide_content(presentation_prompt, output_directory)

Generated Presentation Content:
Here's a detailed outline for a 20-slide presentation in Korean on the Paris 2024 Olympics Closing Ceremony Success Prediction:

1. 제목: 파리 2024 올림픽 폐회식 성공 예측
• 주제 소개
• 발표 개요
• 주요 논점 미리보기

2. 배경: 올림픽 폐회식의 중요성
• 올림픽 정신 마무리
• 문화적 교류의 장
• 개최국의 이미지 제고
• 전 세계 시청자 수: 약 30억 명 (2020 도쿄 올림픽 기준)

3. 파리 2024 올림픽 개요
• 개최 일정: 2024년 7월 26일 - 8월 11일
• 주요 경기장: 에펠탑, 베르사유 궁전 등
• 참가국 수: 약 200개국
• 예상 관중 수: 약 1,300만 명

4. 폐회식 장소: 세느 강
• 혁신적인 야외 폐회식 계획
• 세느 강 6km 구간 활용
• 예상 현장 관객 수: 약 60만 명

5. 문화적 요소 통합
• 프랑스 예술과 문화 showcase
• 세계 문화와의 조화
• 디지털 기술 활용 예정

6. 지속가능성 focus
• 친환경 재료 사용
• 에너지 효율적인 조명
• 탄소 중립 목표

7. 보안 및 안전 대책
• 테러 위협 대비 계획
• 코로나19 대응 방안
• 군경 동원 규모: 약 35,000명 예상

8. 예산 및 경제적 영향
• 폐회식 예산: 약 1억 유로 예상
• 전체 올림픽 예산: 약 72억 유로
• 예상 경제적 효과: 약 104억 유로

9. 시청률 예측
• TV 시청률 예상: 전 세계 인구의 40% 이상
• 디지털 플랫폼 시청자 수 증가 예상
• 소셜 미디어 참여도 증가 전망

10. 결론 및 주요 시사점
• 혁신적인 폐회식 계획의 잠재력
• 문화적 영향력 강조
• 지속가능성에 대한 commitment

11. 폐회식 주요 행사 예측
• 선수단 입장 방식
• 문화 공연 하이라이트
• 차기 올림픽 개최지 소개: 로스앤젤레스 2028

1